# Architecture du projet

On a deux catégories de classes : **la logique du jeu** et **la logique des Solvers (IA)**. 
Dans la logique du jeu, on a la classe `Board` qui s'occupe de tout ce qui concerne l'état du plateau à un instant t et les classes `Jeu` et `JeuContreIA` qui gère l'enchainement de ces differents des états après intervention des joueurs (Humains ou IAs).
Dans la logique des Solvers (IA) on a la classe `Noeud` sur laquelle repose l'exploration de l'arbre des positions possibles avec l'algorithme du minmax et les classes `SolverSmart`, `SolverDumb` et `SolverRandom` qui sont les IAs qui peuvent jouer au morpion et cité selon leur ordre de niveau au jeu, du meilleur au plus faible. 

## Classe Board : 

Attributs : 
- int :`dimension` représente a taille du plateau
- array(array(int)) : `plateau` , le plateau de jeu representé par une liste 2D qui ne peut contenir que 3 élements : **-1** represente une case vide, **1** represente un pion du joueur max et est affiché comme X et **0** represente un pion du joueur min et est affiché comme O.
        
Méthodes :
- int : `add_score()` sert au calcul du score d'un joueur à un instant t. [¹] 
- int : `player_score(player)` retourne le score du joueur, la methode évalue la position des pions du                    `player` (max/1 [²] ou min/0 [³]) ~~passée en paramètre~~ sur l'ensemble du plateau. Plus le                         score est grand, plus la situation est avantageuse pour le `player` en question. 
- int : `game_eval()` retourne la valeur de l'état du plateau à un instant t. Plus sa valeur grande, plus le joueur max             gagne au détriment du joueur min et inversement. 
- string : `to_string` donne une représentation en chaine de caractères utile à la présentation et agréable à voir            pour un joueur humain. 
- array(Board) : `move_neighbors(player)` retourne la liste des Boards voisins pour le prochain coup du `player`              (max/1 [²] ou min/0 [³]).
- boolean : `is_X_winner()`  rends True quand X (le joueur max) gagne et False sinon.
- boolean : `is_O_winner()`  rends True quand O (le joueur min) gagne et False sinon.
- boolean : `is_over()` rends True quand la partie est finie et False sinon.
      

In [1]:
import copy

class Board:
    
    def __init__(self, table):
        self.dimension = 3
        if(len(table) != 3):
            print("Le plateau n'est pas de la bonne taille")
        else :
            self.plateau = copy.deepcopy(table)
            
    def add_score(self, score, aligned_pawns):
        if(aligned_pawns == 0):
            score += 0
        elif(aligned_pawns == 1):
            score += 1
        elif(aligned_pawns == 2):
            score += 10
        elif(aligned_pawns == 3):
            score += 10000
        return score
                    
    def player_score(self, player):
        score = 0
            #k - nombre de pions alignés du joueur sur les lignes
            #l - nombre de pions alignés du joueur sur les colonnes
            #score de la 1ere à la 6e ligne
        for i in range(self.dimension):
            k = 0
            l = 0 
            for j in range(self.dimension):
                if(self.plateau[i][j] == player):
                    k += 1
                if(self.plateau[j][i] == player):
                    l += 1
            score += self.add_score(score, k)
            score += self.add_score(score, l)
            #score sur la 1ere diagonale
        k = 0
        for i in range(self.dimension):
            if(self.plateau[i][i] == player):
                k += 1
        score += self.add_score(score, k)
            #score sur la 2e diagonale
        k = 0
        j = 2
        for i in range(self.dimension):
            if(self.plateau[i][j] == player):
                k += 1
            j -= 1
        score += self.add_score(score, k)
        if(score>= 10000):
            score = 10000
        return score
        
    def game_eval(self):
        return(self.player_score(1) - self.player_score(0))
        
    def to_string(self):
        s = "\n"
        s += "--------------------------\n"
        for i in range(self.dimension):
            for j in range(self.dimension):
                if(self.plateau[i][j] == -1):
                    s += "   \t |"
                elif(self.plateau[i][j] == 1):
                    s += "  X\t |"
                elif(self.plateau[i][j] == 0):
                    s += "  O\t |"
            s += "\n"
            s += "--------------------------\n"
        s += "\n"
        return s
        
    def move_neighbors(self, player):
        l = []
        for i in range(self.dimension):
            for j in range(self.dimension):
                if(self.plateau[i][j] == -1):
                    next_board = Board(self.plateau)
                    next_board.plateau[i][j] = player
                    l.append(next_board)
        return l
        
    def is_X_winner(self):
        if(self.player_score(1) >= 10000):
            return True
        else :
            return False
        
    def is_O_winner(self):
        if(self.player_score(0) >= 10000):
            return True
        else:
            return False
        
    def is_over(self):
        if(self.is_X_winner() or self.is_O_winner()):
            return True
        else :
            for i in range(self.dimension):
                for j in range(self.dimension):
                    if(self.plateau[i][j] == -1):
                        return False
                    else:
                        continue
        return True

## Classe Noeud
Un noeud determine un état du plateau à un moment du jeu donné. On obtient aussi un arbre qu'on peut explorer qui contient également pour chaque Board, le coup précedent et les coups futurs.

Attributs :
- Noeud : `pere` représente le noeud du coup précédent. 
- Board : `board` représente le plateau du coup courant.
- Boolean : `joueur` représente le tour du jour qui doit jouer en conformité avec ce qui a été établi précédemment. (Cf. Doc ou commentaires dans le code).
- int : `nb_fils` représente, pour un joueur donné, le nombre de possibilités à jouer pour le prochain coup.
- array(Noeud) : `fils` est une liste de `Noeud` dans lesquels se trouvent, pour joueur donné, les prochains coups à jouer et toutes les informations supplémentaires relatives à ce coup(valeur, coups suivants etc). 
- int : `value` represente la valeur d'un état du plateau. Pour l'algorithme du minmax, elle sert à mesurer la pertinence d'un coup. 


In [2]:
class Noeud:
    def __init__(self, b, p, joueur):
        self.pere = p
        self.board = b
        self.turn = joueur
        #Si joueur = True, le joueur min a déjà joué et au joueur max de jouer
        #Si joueur = False, le joueur max a déjà joué et au joueur min de jouer
        self.nb_fils = len(b.move_neighbors(int(joueur)))
        self.fils = [0]*self.nb_fils
        if(self.board.is_over() or self.nb_fils == 0):
            self.value = self.board.game_eval()
            
        

Note : Pour les besoins de l'algorithme du minmax, le joueur min (joueur 0) joue toujours en premier pour le reste l'exercice. 

## 4. Classe Solver smart
Une IA qui joue toujours le coup optimal.

Attributs :
- int : `nb_nodes` represente la profondeur de l'arbre du minmax, utile pour les calculs de compléxité. 
- Board : `board_to_solve` plateau de départ sur lequel il faut déterminer le prochain  move. 
- Noeud : `initial_node` est le noeud initial qui contient le plateau à résoudre.
- Boolean : `joueur`permet de differencier au besoin le joueur min(false) du joueur max(true). 
- Board : `next_board`représente le plateau vers lequel nous conduirait notre meilleur coup. 

Méthodes :
- void : `minmax(initial_node)` construit un arbre pour explorer les possibilités des prochains coups à partir d'un noeud donné. Dans notre cas, cet arbre est complet. 
- int : `max_value(node)` demarre la construction de l'arbre pour le joueur max
- int : `min_value(node)` demarre la construction de l'arbre pour le joueur min
- array(Board) : `next_moves` retourne la liste des Boards voisins pour le prochain coup.
- Board : `next_move` retourne le meilleur coup parmi la liste des coups voisins. 

In [3]:
import math

class SolverSmart:
    def __init__(self, board, joueur):
        self.nb_nodes = 0 #profondeur de l'arbre du minmax
        self.board_to_solve = board #plateau à résoudre
        self.initial_node = Noeud(board, None, joueur) #Noeud racine de l'arbre du minmax
        self.joueur = joueur
        self.minmax(self.initial_node)
        self.next_board = self.next_move(self.initial_node)  #prochaine position à jouer
        
    def next_moves(self, node):
        return node.board.move_neighbors(int(node.turn))
    
    def minmax(self, node):
        self.nb_nodes = 0
        if(self.joueur):
            self.max_value(node)
        else:
            self.min_value(node)
        
    def max_value(self, n):
        self.nb_nodes += 1
        j = 0
        if(n.board.is_over() or (n.nb_fils == 0)):
            return n.value
        else:
            for b in self.next_moves(n):
                n.fils[j] = Noeud(b, n, not(n.turn))
                j += 1
        res = -math.inf
        for i in range(n.nb_fils):
            res = max(res, self.min_value(n.fils[i]))
        n.value = res
        return res
    
    def min_value(self, n):
        j = 0
        self.nb_nodes += 1
        if(n.board.is_over() or (n.nb_fils == 0)):
            return n.value
        else:
            for b in self.next_moves(n):
                n.fils[j] = Noeud(b, n, not(n.turn))
                j += 1
        res = math.inf
        for i in range(n.nb_fils):
            res = min(res, self.max_value(n.fils[i]))
        n.value = res
        return res
    
    def next_move(self, node):
        next_board = node.board
        if(node.board.is_over()):
            return node.board
        for n in node.fils:
            if(n.value == node.value):
                next_board = n.board
        return next_board
        

## 2. Classe Solver Random
Une IA qui joue complétement au hasard. Elle hérite de la classe SolverSmart sauf pour `next_move` le calcul du prochain meilleur coup ou c'est un choix aléatoire qui est fait. 

In [4]:
import random

class SolverRandom(SolverSmart):
    def __init__(self, board, joueur):
        SolverSmart.__init__(self, board, joueur)
        self.next_board = self.next_move(self.initial_node)  #prochaine position à jouer
        
    def next_move(self, node):
        next_move = node.board
        if(node.board.is_over()):
            return node.board
        min_val = 0
        max_val = node.nb_fils - 1
        if(max_val - min_val <= 1):
            j = 0
        else :
            j = random.randrange(min_val, max_val)
        n = node.fils[j]
        next_move = n.board
        return next_move

## 3. Classe Solver Dumb

Une IA qui joue parfois de bons coups par fois des coups au hasard. Elle hérite aussi de la classe SolverSmart. Elle fait des choix tantot aléatoires, tantot optimaux. 

In [5]:
import random

class SolverDumb(SolverSmart):
    def __init__(self, board, joueur):
        SolverSmart.__init__(self, board, joueur)
        self.next_board = self.next_move(self.initial_node)  #prochaine position à jouer
        
    def next_move(self, node):
        next_move = node.board
        if(node.board.is_over()):
            return node.board
        
        min_val = 0
        max_val = node.nb_fils - 1
        if(max_val - min_val <= 1):
            j = 0
        else :
            j = random.randrange(min_val, max_val)
        n_dumb = node.fils[j]
        
        for n in node.fils:
            if(n.value == node.value):
                smart_move = n.board
        choice = random.randrange(0,3)
        if(choice == 0):
            next_move = smart_move
        else :
            next_move = n_dumb.board
        return next_move

## 1. Classe Jeu
 
 Une classe Jeu Humain contre Humain. Elle sert de plateforme pourque deux humains puissent jouer l'un contre l'autre.

Attributs :  
- Board : `table_jeu` est le plateau sur lequel va se dérouler le jeu. 
- int : `joueur_min` correspond au premier joueur
- int : `joueur_max`correspond au second joueur
- int : `current_player` correspond au joueur courant

Méthodes :
- void : `game_instructions()` fait de l'affichage en console pour expliquer les règles du jeu et quelques instructions pratiques.
- void : `start_game()` démarre la partie, elle fait aussi de l'affichage pour rendre le jeu un peu plus agréable à l'utilisateur. 

In [6]:
class Jeu:
    def __init__(self):
        #le plateau qu'on va utiliser
        self.table_jeu = Board([
            [-1, -1, -1],
            [-1, -1, -1],
            [-1, -1, -1]])
        self.joueur_min = 0
        self.joueur_max = 1
        self.current_player = self.joueur_min  
            
    def game_instructions(self):
        s = "\n"
        s += "Cette table ci dessous indique les numéros des différentes cases libres sur le plateau en cours de jeu.\n"
        s += "\n"
        s += "--------------------------\n"
        for i in range(self.table_jeu.dimension):
            for j in range(self.table_jeu.dimension):
                if(self.table_jeu.plateau[i][j] == -1):
                    s += ("   " + str(3*i+1+j) + "\t |")
                elif(self.table_jeu.plateau[i][j] == 1):
                    s += "  X\t |"
                elif(self.table_jeu.plateau[i][j] == 0):
                    s += "  O\t |"
            s += "\n"
            s += "--------------------------\n"
        s += "\n"
        s += "Instructions:\n"
        s += "1. Pour jouer, indiquez le numéro de la case ou vous souhaitez placer votre pion.\n"
        s += "2. Faites attention à ne pas sortir de la plage [1,9].\n"
        s += "3. Jouer sur une case déjà prise entraine une pénalité : votre tour sera sauté.\n"
        s += "4. Amusez vous bien au Morpion.\n"
        
        return s
    
    def start_game(self):
        
        print(self.game_instructions())
        
        print("\nVous aller commencer en tant que joueur 0 et votre coup sera représenté par la lettre O.\n")
        print("\nLe prochain joueur sera le joueur 1 et son coup sera représenté par la lettre X.\n ")
        
        while(not(self.table_jeu.is_over())):
            print("*************************************************************\n")
            print("Joueur " + str(self.current_player) + " - Tapez un chiffre entre 1 et 9: \n")
            c = input()
            while (c not in ["1", "2", "3", "4", "5", "6", "7", "8", "9"]) :
                print("Le coup que vous voulez jouer est invalide, relisez les instructions!\n")
                c = input()
            else :
                coup = int(c)
            if(coup == 1):
                if(self.table_jeu.plateau[0][0] == -1):
                    self.table_jeu.plateau[0][0] = self.current_player
                else:
                    print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
            elif(coup == 2):
                if(self.table_jeu.plateau[0][1] == -1):
                    self.table_jeu.plateau[0][1] = self.current_player
                else:
                    print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
            elif(coup == 3):
                if(self.table_jeu.plateau[0][2] == -1):
                    self.table_jeu.plateau[0][2] = self.current_player
                else:
                    print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
            elif(coup == 4):
                if(self.table_jeu.plateau[1][0] == -1):
                    self.table_jeu.plateau[1][0] = self.current_player
                else:
                    print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
            elif(coup == 5):
                if(self.table_jeu.plateau[1][1] == -1):
                    self.table_jeu.plateau[1][1] = self.current_player
                else:
                    print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
            elif(coup == 6):
                if(self.table_jeu.plateau[1][2] == -1):
                    self.table_jeu.plateau[1][2] = self.current_player
                else:
                    print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
            elif(coup == 7):
                if(self.table_jeu.plateau[2][0] == -1):
                    self.table_jeu.plateau[2][0] = self.current_player
                else:
                    print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
            elif(coup == 8):
                if(self.table_jeu.plateau[2][1] == -1):
                    self.table_jeu.plateau[2][1] = self.current_player
                else:
                    print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
            elif(coup == 9):
                if(self.table_jeu.plateau[2][2] == -1):
                    self.table_jeu.plateau[2][2] = self.current_player
                else:
                    print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
            
            print(self.table_jeu.to_string())
            
            if(self.table_jeu.is_O_winner()):
                print("\nFélicitations ! Le joueur " + str(self.current_player) + " a gagné!" )
            elif(not(self.table_jeu.is_X_winner()) and self.table_jeu.is_over()):
                print("\nRéessayer une autre fois ! Vous y êtes presque !")
            
            if(self.current_player == self.joueur_min):
                self.current_player = self.joueur_max
            else:
                self.current_player = self.joueur_min

## Classe JeuHumainContreIA

Une classe qui hérite de la classe Jeu et permets une partie entre un Humain contre une IA.  

In [7]:
class JeuHumainContreIA(Jeu) :
    def __init__(self, solver_type):
        Jeu.__init__(self)
        self.solver_type = solver_type
    
    def start_game(self):
        
        print(self.game_instructions())
        
        print("\nVous aller commencer en tant que joueur 0 et votre coup sera représenté par la lettre O.\n")
        print("\nLe prochain joueur (IA) sera le joueur 1 et son coup sera représenté par la lettre X.\n ")
        
        while(not(self.table_jeu.is_over())):
            if(self.current_player == self.joueur_min):
                print("*************************************************************\n")
                print("Joueur " + str(self.current_player) + " - Tapez un chiffre entre 1 et 9: \n")
                c = input()
                while (c not in ["1", "2", "3", "4", "5", "6", "7", "8", "9"]) :
                    print("Le coup que vous voulez jouer est invalide, relisez les instructions!\n")
                    c = input()
                else :
                    coup = int(c)
                    if(coup == 1):
                        if(self.table_jeu.plateau[0][0] == -1):
                            self.table_jeu.plateau[0][0] = self.current_player
                        else:
                            print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
                    elif(coup == 2):
                        if(self.table_jeu.plateau[0][1] == -1):
                            self.table_jeu.plateau[0][1] = self.current_player
                        else:
                            print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
                    elif(coup == 3):
                        if(self.table_jeu.plateau[0][2] == -1):
                            self.table_jeu.plateau[0][2] = self.current_player
                        else:
                            print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
                    elif(coup == 4):
                        if(self.table_jeu.plateau[1][0] == -1):
                            self.table_jeu.plateau[1][0] = self.current_player
                        else:
                            print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
                    elif(coup == 5):
                        if(self.table_jeu.plateau[1][1] == -1):
                            self.table_jeu.plateau[1][1] = self.current_player
                        else:
                            print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
                    elif(coup == 6):
                        if(self.table_jeu.plateau[1][2] == -1):
                            self.table_jeu.plateau[1][2] = self.current_player
                        else:
                            print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
                    elif(coup == 7):
                        if(self.table_jeu.plateau[2][0] == -1):
                            self.table_jeu.plateau[2][0] = self.current_player
                        else:
                            print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
                    elif(coup == 8):
                        if(self.table_jeu.plateau[2][1] == -1):
                            self.table_jeu.plateau[2][1] = self.current_player
                        else:
                            print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")
                    elif(coup == 9):
                        if(self.table_jeu.plateau[2][2] == -1):
                            self.table_jeu.plateau[2][2] = self.current_player
                        else:
                            print("\nPosition illégalle ! Case déjà occupée ! \nPénalité - Vous sautez le tour !")

                print(self.table_jeu.to_string())

                if(self.table_jeu.is_O_winner()):
                    print("\nFélicitations ! Le joueur " + str(self.current_player) + " a gagné!" )
                elif(not(self.table_jeu.is_X_winner()) and self.table_jeu.is_over()):
                    print("\nRéessayer une autre fois ! Vous y êtes presque !")
                
                if(self.current_player == self.joueur_min):
                    self.current_player = self.joueur_max
                else:
                    self.current_player = self.joueur_min
                
            else :
                print("Joueur " + str(self.current_player) + " - AUTO \n")
                if(self.solver_type == 0):
                    solver = SolverSmart(self.table_jeu, True)
                elif(self.solver_type == 1):
                    solver = SolverRandom(self.table_jeu, True)
                elif(self.solver_type == 2):
                    solver = SolverDumb(self.table_jeu, True)
                    
                self.table_jeu = solver.next_board
                print(self.table_jeu.to_string())
                if(self.table_jeu.is_X_winner()):
                    print("Dommage ! Le joueur " + str(self.current_player) + " a gagné!")
                elif(not(self.table_jeu.is_O_winner()) and self.table_jeu.is_over()):
                    print("Réessayer une autre fois ! Vous y êtes presque !")
                self.current_player = self.joueur_min
    

## Classe JeuIAContreIA

Un jeu qui permets d'opposer deux IA. Elle hérite également de la classe Jeu. 

In [8]:
class JeuIAContreIA(Jeu) :
    def __init__(self, solver_type_0, solver_type_1):
        Jeu.__init__(self)
        self.solver_type_0 = solver_type_0
        self.solver_type_1 = solver_type_1
    
    def start_game(self):
        
        while(not(self.table_jeu.is_over())):
            
            if(self.current_player == self.joueur_min):
                
                #print("Joueur " + str(self.current_player) + " - AUTO \n")
                if(self.solver_type_0 == 0):
                    solver = SolverSmart(self.table_jeu, False)
                elif(self.solver_type_0 == 1):
                    solver = SolverRandom(self.table_jeu, False)
                elif(self.solver_type_0 == 2):
                    solver = SolverDumb(self.table_jeu, False)
                    
                self.table_jeu = solver.next_board
                #print(self.table_jeu.to_string())
                if(self.table_jeu.is_O_winner()):
                    return self.solver_type_0
                elif(not(self.table_jeu.is_X_winner()) and self.table_jeu.is_over()):
                    return -1
                
                if(self.current_player == self.joueur_min):
                    self.current_player = self.joueur_max
                else:
                    self.current_player = self.joueur_min
                
            else :
                #print("Joueur " + str(self.current_player) + " - AUTO \n")
                if(self.solver_type_1 == 0):
                    solver = SolverSmart(self.table_jeu, True)
                elif(self.solver_type_1 == 1):
                    solver = SolverRandom(self.table_jeu, True)
                elif(self.solver_type_1 == 2):
                    solver = SolverDumb(self.table_jeu, True)
                    
                self.table_jeu = solver.next_board
                #print(self.table_jeu.to_string())
                if(self.table_jeu.is_X_winner()):
                    return self.solver_type_1
                elif(not(self.table_jeu.is_O_winner()) and self.table_jeu.is_over()):
                    return -1
                self.current_player = self.joueur_min

### 5. Simulation de 1000 parties jouées par les IAs les unes contre les autres. 
Chaque IA jouera un nombre determiné de fois contre les autres IA afin qu'on puisse observer leur performances sur un plus grand echantillon de parties jouées. 

In [9]:
#N_PARTIES = 1000, Cette instruction est commentée car son execution prends enormément de temps (des heures).
N_PARTIES = 1000  #Cette instruction peut toujours être altérée pour gagner en temps. 

jeux_resultats_0_vs_0 = []
jeux_resultats_0_vs_1 = []
jeux_resultats_0_vs_2 = []
jeux_resultats_1_vs_0 = []
jeux_resultats_1_vs_1 = []
jeux_resultats_1_vs_2 = []
jeux_resultats_2_vs_0 = []
jeux_resultats_2_vs_1 = []
jeux_resultats_2_vs_2 = []

for i in range(N_PARTIES):
    jeux_resultats_0_vs_0.append(JeuIAContreIA(0,0).start_game())
for i in range(N_PARTIES):
    jeux_resultats_0_vs_1.append(JeuIAContreIA(0,1).start_game())
for i in range(N_PARTIES):
    jeux_resultats_0_vs_2.append(JeuIAContreIA(0,2).start_game())
for i in range(N_PARTIES):
    jeux_resultats_1_vs_0.append(JeuIAContreIA(1,0).start_game())
for i in range(N_PARTIES):
    jeux_resultats_1_vs_1.append(JeuIAContreIA(1,1).start_game())
for i in range(N_PARTIES):
    jeux_resultats_1_vs_2.append(JeuIAContreIA(1,2).start_game())
for i in range(N_PARTIES):
    jeux_resultats_2_vs_0.append(JeuIAContreIA(2,0).start_game())
for i in range(N_PARTIES):
    jeux_resultats_2_vs_1.append(JeuIAContreIA(2,1).start_game())
for i in range(N_PARTIES):
    jeux_resultats_2_vs_2.append(JeuIAContreIA(2,2).start_game())


KeyboardInterrupt: 

### 6. Pourcentage de parties gagnées

Statistiques sur la simulation de parties des IAs. On a pour chacune d'entre elle, le pourcentage de parties gagnées contre elle même, et contre d'autres IA dans le cas ou elle joue en premier et dans le cas contraire. On pourra donc éventuellement voir a quel point le fait de jouer en premier influe sur les performances de ces différentes IAs. 

In [ ]:
score_0_vs_0 = 0
for i in range(len(jeux_resultats_0_vs_0)):
    if(jeux_resultats_0_vs_0[i] == 0):
        score_0_vs_0 += 1 
score_0_vs_0_in_percent = (score_0_vs_0 * 100) / N_PARTIES

score_0_vs_1 = 0
for i in range(len(jeux_resultats_0_vs_1)):
    if(jeux_resultats_0_vs_1[i] == 0):
        score_0_vs_1 += 1 
score_0_vs_1_in_percent = (score_0_vs_1 * 100) / N_PARTIES

score_0_vs_2 = 0
for i in range(len(jeux_resultats_0_vs_2)):
    if(jeux_resultats_0_vs_2[i] == 0):
        score_0_vs_2 += 1
score_0_vs_2_in_percent = (score_0_vs_2 * 100) / N_PARTIES

score_1_vs_0 = 0
for i in range(len(jeux_resultats_1_vs_0)):
    if(jeux_resultats_1_vs_0[i] == 1):
        score_0_vs_1 += 1
score_1_vs_0_in_percent = (score_1_vs_0 * 100) / N_PARTIES

score_1_vs_1 = 0
for i in range(len(jeux_resultats_1_vs_1)):
    if(jeux_resultats_1_vs_1[i] == 1):
        score_1_vs_1 += 1 
score_1_vs_1_in_percent = (score_1_vs_1 * 100) / N_PARTIES

score_1_vs_2 = 0
for i in range(len(jeux_resultats_1_vs_2)):
    if(jeux_resultats_1_vs_2[i] == 1):
        score_1_vs_2 += 1 
score_1_vs_2_in_percent = (score_1_vs_2 * 100) / N_PARTIES

score_2_vs_0 = 0
for i in range(len(jeux_resultats_2_vs_0)):
    if(jeux_resultats_2_vs_0[i] == 2):
        score_2_vs_0 += 1
score_2_vs_0_in_percent = (score_2_vs_0 * 100) / N_PARTIES

score_2_vs_1 = 0
for i in range(len(jeux_resultats_2_vs_1)):
    if(jeux_resultats_2_vs_1[i] == 2):
        score_2_vs_1 += 1
score_2_vs_1_in_percent = (score_2_vs_1 * 100) / N_PARTIES
        
score_2_vs_2 = 0
for i in range(len(jeux_resultats_2_vs_2)):
    if(jeux_resultats_2_vs_2[i] == 2):
        score_2_vs_2 += 1
score_2_vs_2_in_percent = (score_2_vs_2 * 100) / N_PARTIES
        
print(f"SolverSmart en tant que premier joueur : {score_0_vs_0} parties gagnées contre SolverSmart sur un total de {N_PARTIES} - {'{:.2f}'.format(score_0_vs_0_in_percent)}")
print("\n")
print(f"SolverSmart en tant que premier joueur : {score_0_vs_1} parties gagnées contre SolverRandom sur un total de {N_PARTIES} - {'{:.2f}'.format(score_0_vs_1_in_percent)}")
print("\n")
print(f"SolverSmart en tant que premier joueur : {score_0_vs_2} parties gagnées contre SolverDumb sur un total de {N_PARTIES} - {'{:.2f}'.format(score_0_vs_2_in_percent)}")
print("\n")  
print(f"SolverRandom en tant que premier joueur : {score_1_vs_0} parties gagnées contre SolverSmart sur un total de {N_PARTIES} - {'{:.2f}'.format(score_1_vs_0_in_percent)}")
print("\n")  
print(f"SolverRandom en tant que premier joueur : {score_1_vs_1} parties gagnées contre SolverRandom sur un total de {N_PARTIES} - {'{:.2f}'.format(score_1_vs_1_in_percent)}")
print("\n") 
print(f"SolverRandom en tant que premier joueur : {score_1_vs_2} parties gagnées contre SolverDumb sur un total de {N_PARTIES} - {'{:.2f}'.format(score_1_vs_2_in_percent)}")
print("\n") 
print(f"SolverDumb en tant que premier joueur : {score_2_vs_0} parties gagnées contre SolverSmart sur un total de {N_PARTIES} - {'{:.2f}'.format(score_2_vs_0_in_percent)}")
print("\n") 
print(f"SolverDumb en tant que premier joueur : {score_2_vs_1} parties gagnées contre SolverRandom sur un total de {N_PARTIES} - {'{:.2f}'.format(score_2_vs_1_in_percent)}")
print("\n") 
print(f"SolverDumb en tant que premier joueur : {score_2_vs_2} parties gagnées contre SolverDumb sur un total de {N_PARTIES} - {'{:.2f}'.format(score_2_vs_2_in_percent)}")
print("\n") 


## Graphes des performances des IAs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

stats_ia = {"IaSmart": [], "IaDumb": [], "IaRandom": []}


for N_PARTIES in range(1, 1000):
    for Ia, ia_name in zip([IaSmart, IaDumb, IaRandom], ["IaSmart", "IaDumb", "IaRandom"]):
        stats_ia[ia_name].append(score_in_percent)

### Petit script pour jouer sans encombres

In [ ]:
print("Si vous voulez jouer contre une IA, veuillez renseigner le niveau de difficulté :\n")
print("\t 0 - DIFFICILE - Vous jouerez contre SolverSmart\n")
print("\t 1 - FACILE - Vous jouerez contre SolverRandom\n")
print("\t 2 - MOYEN - Vous jouerez contre SolverDumb\n")    
print("Si vous voulez jouer contre un humain, tapez 3.\n")
choix = input()
mode_de_jeu = int(choix)
while(mode_de_jeu not in range(0,4)):
    print("Votre saisie est invalide, veuillez choisir parmi les options données.\n")
    choix = input()
    mode_de_jeu = int(choix)
else:
    if(mode_de_jeu == 3):
        jeu = Jeu()
        jeu.start_game()
        print("Voulez vous rejouer une partie, tapez 0 pour OUI ou 1 pour NON!\n")
        retry = input()
        while (int(retry) == 0):
            nouveau_jeu = Jeu()
            nouveau_jeu.start_game()
            print("Voulez vous rejouer une partie, tapez 0 pour OUI ou 1 pour NON!\n")
            retry = input()
        else:
            print("À bientôt!!\n")
    else:
        jeu = JeuHumainContreIA(mode_de_jeu)
        jeu.start_game()
        print("Voulez vous rejouer une partie, tapez 0 pour OUI ou 1 pour NON!\n")
        retry = input()
        while (int(retry) == 0):
            nouveau_jeu = JeuHumainContreIA(mode_de_jeu)
            nouveau_jeu.start_game()
            print("Voulez vous rejouer une partie, tapez 0 pour OUI ou 1 pour NON!\n")
            retry = input()
        else:
            print("À bientôt!!\n")

Si vous voulez jouer contre une IA, veuillez renseigner le niveau de difficulté :

	 0 - DIFFICILE - Vous jouerez contre SolverSmart

	 1 - FACILE - Vous jouerez contre SolverRandom

	 2 - MOYEN - Vous jouerez contre SolverDumb

Si vous voulez jouer contre un humain, tapez 3.

0

Cette table ci dessous indique les numéros des différentes cases libres sur le plateau en cours de jeu.

--------------------------
   1	 |   2	 |   3	 |
--------------------------
   4	 |   5	 |   6	 |
--------------------------
   7	 |   8	 |   9	 |
--------------------------

Instructions:
1. Pour jouer, indiquez le numéro de la case ou vous souhaitez placer votre pion.
2. Faites attention à ne pas sortir de la plage [1,9].
3. Jouer sur une case déjà prise entraine une pénalité : votre tour sera sauté.
4. Amusez vous bien au Morpion.


Vous aller commencer en tant que joueur 0 et votre coup sera représenté par la lettre O.


Le prochain joueur (IA) sera le joueur 1 et son coup sera représenté par la lettre 

4

--------------------------
  X	 |  O	 |  X	 |
--------------------------
  O	 |  O	 |   	 |
--------------------------
  O	 |  X	 |   	 |
--------------------------


Joueur 1 - AUTO 


--------------------------
  X	 |  O	 |  X	 |
--------------------------
  O	 |  O	 |  X	 |
--------------------------
  O	 |  X	 |   	 |
--------------------------


*************************************************************

Joueur 0 - Tapez un chiffre entre 1 et 9: 

9

--------------------------
  X	 |  O	 |  X	 |
--------------------------
  O	 |  O	 |  X	 |
--------------------------
  O	 |  X	 |  O	 |
--------------------------



Réessayer une autre fois ! Vous y êtes presque !
Voulez vous rejouer une partie, tapez 0 pour OUI ou 1 pour NON!

0

Cette table ci dessous indique les numéros des différentes cases libres sur le plateau en cours de jeu.

--------------------------
   1	 |   2	 |   3	 |
--------------------------
   4	 |   5	 |   6	 |
--------------------------
   7	 |   8	 |   9	

[¹]: Il s'agit d'un refacto pour eviter de la répétition de code dans le calcul du score `player_score(player)`.

[²]: Au cours de l'ensemble du projet, le joueur min ou premier joueur a toujours été codifié en 0 et [³] le joueur max en 1.